# Projet maison n° 4

In [7]:
# imports
import requests
from bs4 import BeautifulSoup
import re

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [145]:

# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"


def get_prices_from_url(url):
    r=requests.get(url)
    prices = {}
    rows = []
    # Conversion du contenu 
    content = r.content.decode('utf-8')
    #print(content)
    soup = BeautifulSoup(content)
    # Recherche des éléments de la balise h2
    for tag in soup.findAll('div', attrs={'class': 'pricing-tables pure-g'}):
        titres = []
        for element in tag.findAll('h2'):
            titres.append(element.text) # Ajout de chaque élément dans la liste titres
    #print(titres)
    price=[]
    # extraction avec regex des prix
    rx1 = re.compile('<span class="pricing-table-price">([^<]+)<span>')
    for match in rx1.finditer(content):
        price.append((match.group(1))[2:])

    storage=[]
    # extraction avec regex du stockage
    rx2 = re.compile('<li>([0-9]+[GBT]+)[^a-z][^<]+</li>')
    for match2 in rx2.finditer(content):
        storage.append((match2.group(1)))

    databases=[]
    # extraction avec regex du nombre de base de données
    rx3 = re.compile('<li>([0-9]+) database?[^<]+</li>')
    for match3 in rx3.finditer(content):
        databases.append((match3.group(1)))

    for i in range(len(titres)):
        # Structuration des données dans le dictionnaire prices
        data=dict(zip(['price','storage','databases'],[price[i].strip(),storage[i],int(databases[i])]))
        prices[titres[i]]= data
    
    return prices


#get_prices_from_url(URL_PAGE2)
#get_prices_from_url(URL_PAGE3)

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [370]:
# partie B-1
import re
import pandas as pd
import numpy as np
url='https://www.beerwulf.com/fr-fr/p/bieres/ciney-blond.25?scrollTo=relatedProducts'#'https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33'#'https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille'

        

    
def extract_beer_infos(url):
    
    r=requests.get(url)

    content = r.content.decode('utf-8')

    soup = BeautifulSoup(content, "lxml")
    #print(content)
    #soup

    nom=[]
    degre = []
    style=[]
    contenu=[]
    origine=[]
    brasseur=[]

    for tag in soup.findAll('div',attrs={'class':"product-detail-info-title"}):
        n=tag.find('h1')
        nom.append(n.text)


    for tag in soup.findAll('div', attrs={'class':"small-12 content-column"}):
        for tag2 in tag.findAll('div',attrs={'class':"product-info beer-product-info"}):

            for c in tag2.findAll('dd',{'class':"small-6 medium-9 columns js-beer-volume"}):
                contenu.append(int((c.text.strip())[0:2]))

            for de in tag2.findAll('dd',{'class':"small-6 medium-9 columns"}):
                val=re.search('([0-9]+,[0-9]+)%', de.text.strip())
                if val:
                    d=str(val.group(1)).replace(',','.')
                    degre.append(float(d[0:4]))
            for ori in tag2.findAll('dd',{'class':"small-6 medium-9 columns js-beer-country"}):
                origine.append(ori.text.strip())
            data=[]
            for bra in tag2.findAll('dd'):
                value=bra.text
                data.append(value)
            style.append(data[0].strip())
            
    beer_infos = {
        'Nom': nom[0],
        'Style': style[0],
        'Contenu': contenu[0],
        'Degré d’alcool': degre[0],
        'Origine': origine[0],
        'Brasseur': data[4].strip()
    }
    
    
    return beer_infos
extract_beer_infos(url)

{'Nom': 'Ciney Blonde',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Ciney'}

In [371]:
# partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France"#&container=Bouteille"

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    r=requests.get(url)
    data1=r.json()
    
    for element in data1['items']:
        beer_pages.append(element['contentReference'])
    # Sequential version (slow):
    beers = []
    for link in beer_pages:    
        
        beers.append(extract_beer_infos('https://www.beerwulf.com/' + link))
    # Facultatif
    # Parallel version (faster):
    # beers = []
    
    return beers

#extract_beer_list_infos(URL_BEERLIST_FRANCE)

In [373]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [374]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 2.697s

OK
